## 1. Importing the Libraries

In [72]:
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## 2. Reading the Data

In [73]:
PROJECT_DIR= Path(r"F:\Flight-Price-Prediction")
DATA_DIR = "data"

In [74]:
def get_data(name):
    file_name = f"{name}.csv"
    file_path = PROJECT_DIR/DATA_DIR/file_name
    return pd.read_csv(file_path)

In [75]:
flights = get_data('flights')

In [76]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- This dataset contains 10,683 rows and 11 columns
- Columns `Route` and `Total_Stops` have one missing value each
- Data Types of some features aren't correct

## 3. Preliminary Analysis

### 3.1 Check data types

In [6]:
flights.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [7]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

In [8]:
type(flights.Date_of_Journey.iloc[0])

str

In [9]:
type(flights.Dep_Time.iloc[0])

str

In [10]:
type(flights.Arrival_Time.iloc[0])

str

### 3.2 Check duplicates

In [11]:
(
    flights
    .duplicated()
    .sum()
)

np.int64(220)

In [12]:
(
    flights
    .loc[flights.duplicated(keep = False)]
    .sort_values(['Airline','Date_of_Journey','Source','Destination'])
)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


### 3.3 Observations

- The type of `Date_of_Journey`,`Dep_Time` and `Arrival_Time` should be changed to datetime
- The type of `duration` and `Total_Stops` is mixed. It should be numerical
- There are 220 duplicates these should be removed

## 4. Detailed Analysis

### 4.1 `Airline`

In [13]:
flights.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

- Some of the values are inconsistent and inaccurate

In [14]:
(
    flights
    .Airline
    .str.replace(" Premium economy","")
    .str.replace(" Business","")
    .str.title()
    .unique()
)

array(['Indigo', 'Air India', 'Jet Airways', 'Spicejet',
       'Multiple Carriers', 'Goair', 'Vistara', 'Air Asia', 'Trujet'],
      dtype=object)

### 4.2 `Date_of_Journey`

In [15]:
flights.Date_of_Journey

0        24/03/2019
1         1/05/2019
2         9/06/2019
3        12/05/2019
4        01/03/2019
            ...    
10678     9/04/2019
10679    27/04/2019
10680    27/04/2019
10681    01/03/2019
10682     9/05/2019
Name: Date_of_Journey, Length: 10683, dtype: object

In [16]:
pd.to_datetime(flights.Date_of_Journey, dayfirst = True,format = 'mixed')

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

### 4.3 `Source` and `Destination`

In [17]:
flights.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

In [18]:
(
    flights
    .Source
    .str.replace("Banglore","Bangalore")
)

0        Bangalore
1          Kolkata
2            Delhi
3          Kolkata
4        Bangalore
           ...    
10678      Kolkata
10679      Kolkata
10680    Bangalore
10681    Bangalore
10682        Delhi
Name: Source, Length: 10683, dtype: object

In [19]:
flights.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

In [20]:
(
    flights
    .Destination
    .str.replace("Banglore","Bangalore")
    .str.replace("New Delhi","Delhi")
)

0            Delhi
1        Bangalore
2           Cochin
3        Bangalore
4            Delhi
           ...    
10678    Bangalore
10679    Bangalore
10680        Delhi
10681        Delhi
10682       Cochin
Name: Destination, Length: 10683, dtype: object

### 4.4 `Dep_Time` and `Arrival_Time`

In [21]:
flights.Dep_Time

0        22:20
1        05:50
2        09:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680    08:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: object

In [22]:
(
    flights
    .Dep_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
)

Series([], Name: Dep_Time, dtype: object)

In [23]:
pd.to_datetime(flights.Dep_Time,dayfirst = True,format = 'mixed').dt.time

0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

In [24]:
flights.Arrival_Time

0        01:10 22 Mar
1               13:15
2        04:25 10 Jun
3               23:30
4               21:35
             ...     
10678           22:25
10679           23:20
10680           11:20
10681           14:10
10682           19:15
Name: Arrival_Time, Length: 10683, dtype: object

In [25]:
(
    flights
    .Arrival_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
    .str.split(" ", n=1)
    .str.get(1)
    .unique()
)

array(['22 Mar', '10 Jun', '13 Mar', '02 Mar', '10 May', '04 Mar',
       '13 Jun', '28 May', '19 Mar', '07 May', '02 Jun', '16 Jun',
       '19 May', '16 May', '28 Jun', '02 May', '28 Mar', '19 Jun',
       '04 Apr', '25 Mar', '07 Mar', '25 Jun', '07 Jun', '25 May',
       '13 May', '16 Mar', '22 May', '10 Apr', '04 Jun', '20 May',
       '28 Apr', '25 Apr', '10 Mar', '19 Apr', '13 Apr', '02 Apr',
       '23 Mar', '22 Apr', '11 May', '07 Apr', '03 May', '08 Mar',
       '03 Mar', '05 Mar', '22 Jun', '04 May', '26 May', '16 Apr',
       '26 Jun', '29 May', '29 Jun', '29 Mar', '23 May', '17 Jun'],
      dtype=object)

### 4.5 `Duration`

In [26]:
flights.Duration

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
          ...  
10678    2h 30m
10679    2h 35m
10680        3h
10681    2h 40m
10682    8h 20m
Name: Duration, Length: 10683, dtype: object

In [27]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains("m")]
)

2        19h
18       23h
33       22h
44       12h
53        3h
        ... 
10591    23h
10638    14h
10639    38h
10673    15h
10680     3h
Name: Duration, Length: 1031, dtype: object

In [28]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains("h")]
)

6474    5m
Name: Duration, dtype: object

In [29]:
(
    flights
    .Duration
    .drop(index = [6474])
    .str.split(" ",expand=True)
    .set_axis(["hours","minutes"],axis=1)
    .assign(
        hours = lambda df_: (
            df_
            .hours
            .str.replace("h","")
            .astype('int')
            .mul(60)
        ),
        minutes = lambda df_:(
            df_
            .minutes
            .str.replace("m","")
            .fillna("0")
            .astype('int')
        )
    )
    .sum(axis=1)
    .rename('duration_minutes')
    .to_frame()
    .join(flights.Duration.drop(index=[6474]))
)

,duration_minutes,Duration
0,170,2h 50m
1,445,7h 25m
2,1140,19h
3,325,5h 25m
4,285,4h 45m
...,...,...
10678,150,2h 30m
10679,155,2h 35m
10680,180,3h
10681,160,2h 40m


In [30]:
flights.iloc[[6474]]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,6/03/2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


- The observation indexed `6474` has clearly mentioned the duration wrong so it will be removed

### 4.6 `Total_Stops`

In [37]:
flights.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [47]:
(
    flights
    .Total_Stops
    .replace("non-stop","0")
    .str.replace(" stops?","",regex=True)
    .pipe(lambda ser: pd.to_numeric(ser))
)

0        0.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
10678    0.0
10679    0.0
10680    0.0
10681    0.0
10682    2.0
Name: Total_Stops, Length: 10683, dtype: float64

### 4.7 `Additional_Info`

In [53]:
flights.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

## 5. Cleaning Operations

In [77]:
def convert_to_minutes(ser):
    return (
        ser
        .str.split(" ",expand=True)
        .set_axis(["hours","minutes"],axis=1)
        .assign(
        hours = lambda df_: (
                df_
                .hours
                .str.replace("h","")
                .astype('int')
                .mul(60)
            ),
        minutes = lambda df_:(
                df_
                .minutes
                .str.replace("m","")
                .fillna("0")
                .astype('int')
            )
        )
        .sum(axis=1)
    )

In [83]:
def clean_data(df):
    return (
        df
        .drop(index = [6474])
        .assign(
            **{
                col: df[col].str.strip()
                for col in df.select_dtypes(include = "O").columns
            }
        )
        .rename(columns = str.lower)
        .assign(
            airline = lambda df_: (
                                      df_
                                      .airline
                                      .str.replace(" Premium economy","")
                                      .str.replace(" Business","")
                                      .str.title()                        
            ),
            date_of_journey = lambda df_: pd.to_datetime(df_.date_of_journey,dayfirst = True,format = 'mixed'),
            source = lambda df_: (
                df_
                .source
                .str.replace("Banglore","Bangalore")
            ),
            destination = lambda df_: (
                df_
                .destination
                .str.replace("Banglore","Bangalore")
                .str.replace("New Delhi","Delhi")
            ),
            dep_time = lambda df_: pd.to_datetime(df_.dep_time,dayfirst = True,format = 'mixed').dt.time,
            arrival_time = lambda df_: pd.to_datetime(df_.arrival_time,dayfirst = True,format = 'mixed').dt.time,
            duration = lambda df_: df_.duration.pipe(convert_to_minutes),
            total_stops = lambda df_: (
                df_
                .total_stops
                .replace("non-stop","0")
                .str.replace(" stops?","",regex=True)
                .pipe(lambda ser: pd.to_numeric(ser))
            ),
           additional_info = lambda df_: df_.additional_info.str.replace("No info","No Info")
        )
        .drop(columns = ['route'])
    )

In [84]:
flights_cleaned = clean_data(flights)

## 6. Split the Data

In [58]:
X = flights_cleaned.drop(columns = ['price'])
y = flights_cleaned.price.copy()

In [62]:
X_,X_test,y_,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train,X_val,y_train,y_val = train_test_split(X_,y_,test_size=0.2,random_state=42)

print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)
print(X_val.shape,y_val.shape)

(6836, 9) (6836,)
(2137, 9) (2137,)
(1709, 9) (1709,)


## 7. Export the Subsets

In [68]:
def export_dataset(X,y,name):
    file_name = f"{name}.csv"
    file_path = PROJECT_DIR/DATA_DIR/f"{name}"/file_name

    file_path.parent.mkdir(parents=True, exist_ok=True)

    X.join(y).to_csv(file_path,index=False)

    return pd.read_csv(file_path).head()

In [69]:
export_dataset(X_train,y_train,"train")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-03,Delhi,Cochin,02:15:00,04:25:00,1570,1.0,No Info,17024
1,Vistara,2019-03-24,Kolkata,Bangalore,07:10:00,18:45:00,695,1.0,No Info,16932
2,Spicejet,2019-04-09,Bangalore,Delhi,09:30:00,12:20:00,170,0.0,No Info,4423
3,Indigo,2019-04-27,Bangalore,Delhi,21:15:00,00:15:00,180,0.0,No Info,3943
4,Air India,2019-06-12,Delhi,Cochin,09:45:00,09:25:00,1420,1.0,No Info,7480


In [70]:
export_dataset(X_test,y_test,"test")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Multiple Carriers,2019-05-21,Delhi,Cochin,15:05:00,01:30:00,625,2.0,No Info,16655
1,Goair,2019-06-03,Delhi,Cochin,10:35:00,19:35:00,540,1.0,No Info,4959
2,Vistara,2019-05-09,Kolkata,Bangalore,20:20:00,09:05:00,765,1.0,No Info,9187
3,Indigo,2019-05-24,Chennai,Kolkata,14:45:00,17:05:00,140,0.0,No Info,3858
4,Jet Airways,2019-05-21,Delhi,Cochin,22:50:00,04:25:00,335,1.0,In-flight meal not included,12898


In [71]:
export_dataset(X_val,y_val,"val")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-06-12,Kolkata,Bangalore,16:30:00,12:00:00,1170,1.0,In-flight meal not included,8529
1,Jet Airways,2019-05-18,Bangalore,Delhi,11:10:00,14:05:00,175,0.0,In-flight meal not included,5198
2,Multiple Carriers,2019-06-12,Delhi,Cochin,08:45:00,19:00:00,615,1.0,No Info,11789
3,Multiple Carriers,2019-04-24,Delhi,Cochin,07:10:00,16:10:00,540,1.0,In-flight meal not included,6093
4,Jet Airways,2019-03-27,Delhi,Cochin,15:05:00,04:25:00,800,1.0,No Info,12242
